<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [46]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [47]:
if IN_COLAB:
    %pip install openai
    %pip install requests

if IN_COLAB:
    import os
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "GHPT_Instructor"

### load files into Colab and install package

NOTE: you may need to refresh your Colab files directory to see changes

In [48]:
if IN_COLAB:
    # remove the existing directory
    import shutil
    shutil.rmtree('/content/GHPT-experiments/', ignore_errors=True)
    %git clone "https://github.com/samgregson/GHPT-experiments"
    %pip install -e /content/GHPT-experiments/
    # add the modules to the search path
    import site
    site.main()

Load the correct SSL certificates

In [49]:
if not IN_COLAB:
    from dotenv import load_dotenv
    import os
    import ssl
    load_dotenv()
    context = ssl.create_default_context(cafile=os.environ.get("REQUESTS_CA_BUNDLE"))
    print(os.environ.get("REQUESTS_CA_BUNDLE"))

None


test SSL certificate, this should return `<Response [200]>` if not try restarting the Jupyter kernel

In [50]:
import requests
requests.get("https://api.smith.langchain.com/docs")

<Response [200]>

### OpenAI

In [51]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Custom wrap for VSCode, needs to be the first wrap!
if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

# GHPT

### Import prompts

In [52]:
from prompts.generate_script import system_prompt, prompt_template

# replace all { with {{ and } with }} to escape the curly braces
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

# Instructor

In [53]:
if IN_COLAB:
    !pip install -U instructor
    !pip install anthropic

In [54]:
import instructor

client_instructor = instructor.patch(client, mode=instructor.Mode.TOOLS)

In [55]:
from langsmith import traceable
from models.models import GrasshopperScriptModel

# create the openai api call function
@traceable
def call_openai_instructor(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    completion = client_instructor.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_model=GrasshopperScriptModel,
        max_retries = 2
    )

    return completion

In [56]:
print(system_prompt)


You are a Grasshopper Expert and are going to help create Grasshopper Definitions.
Keep the answers short and concise.
Make sure you create and connect a component for every non-optional input
If you're not sure about the answer, but think there's additional information that could help you, please ask for that information.

Always use the given format, avoid any devitation.



In [67]:
user_prompt = 'Create a cone with a base radius of 5 and a height of 10'
problem_definition_prompt = """given the following description of a grasshopper script below, describe what inputs and outputs are expected for the script. 
Description: {user_prompt}"""

In [68]:
print(problem_definition_prompt.format(user_prompt=user_prompt))

given the following description of a grasshopper script below, describe what inputs and outputs are expected for the script. 
Description: Create a cone with a base radius of 5 and a height of 10


In [69]:
model = "gpt-3.5-turbo-1106"
# model = "gpt-4-1106-preview"
response = call_openai_instructor(prompt=problem_definition_prompt.format(user_prompt=user_prompt), system_prompt=system_prompt)

In [70]:
print(response.json())
initial_response = response.json()

{"ChainOfThought":"Create a cone using the Cone component with a base radius of 5 and a height of 10.","Additions":[{"Name":"Cone","Id":1},{"Name":"Number Slider","Id":2},{"Name":"Number Slider","Id":3}],"Connections":[{"From":{"Id":2,"ParameterName":"Slider"},"To":{"Id":1,"ParameterName":"Radius"}},{"From":{"Id":3,"ParameterName":"Slider"},"To":{"Id":1,"ParameterName":"Height"}}],"Advice":"Adjust the sliders to change the base radius and height of the cone."}


In [61]:
next_prompt = """given the following description of a grasshopper script below and a response (in json form) that you generated about the inputs and outputs, provide the key grasshopper components that would be used to create the script. 
Description: {user_prompt}
Response: {initial_response}"""



In [62]:
response = call_openai_instructor(prompt=next_prompt.format(user_prompt=user_prompt, initial_response=initial_response), system_prompt=system_prompt)

In [63]:
print(response.json())
second_response = response.json()

{"ChainOfThought":"Create a cone using the Cone component with a base radius of 5 and a height of 10.","Additions":[{"Name":"Cone","Id":1},{"Name":"Number Slider","Id":2},{"Name":"Number Slider","Id":3}],"Connections":[{"From":{"Id":2,"ParameterName":"Slider"},"To":{"Id":1,"ParameterName":"Radius"}},{"From":{"Id":3,"ParameterName":"Slider"},"To":{"Id":1,"ParameterName":"Height"}}],"Advice":"Adjust the sliders to change the base radius and height of the cone."}


In [64]:
#TODO check to ensure inputs, outputs, and components exist
#IE for cone, its looking for length, but the actual parameter is height (can do some fuzzy string matching here)

#TODO check to ensure elements are connected properly

In [65]:
#Allow the user to provide manual feedback

feedback = "you forgot to set the values of the number sliders, please adjust the output"
#note - instructor doesnt allow a value field??? 

feedback_prompt = """You generated the previous output {second_response}
but the user has some feedback on that: {feedback}"""

In [66]:
response = call_openai_instructor(prompt=feedback_prompt.format(second_response=second_response, feedback=feedback), system_prompt=system_prompt)
print(response.json())

{"ChainOfThought":"Create a cone using the Cone component with a base radius of 5 and a height of 10.","Additions":[{"Name":"Cone","Id":1},{"Name":"Number Slider","Id":2},{"Name":"Number Slider","Id":3}],"Connections":[{"From":{"Id":2,"ParameterName":"Slider"},"To":{"Id":1,"ParameterName":"Radius"}},{"From":{"Id":3,"ParameterName":"Slider"},"To":{"Id":1,"ParameterName":"Height"}}],"Advice":"Adjust the sliders to change the base radius and height of the cone."}
